In [1]:
import pandas as pd

In [27]:
def cleanData(df,substance):
    df = df.drop(columns=['historianTagnummer'])
    df.rename(columns={'hstWaarde': substance + 'Value'}, inplace=True)
    df[substance + 'Value'] = df[substance + 'Value'].astype(float)
    # finalTotal = finalTotal.drop(columns=['DateTime'])
    df = df.drop(columns=['datumBeginMeting'])
    # df['datumBeginMeting'] = pd.to_datetime(df['datumBeginMeting'])
    # df['StartDate'] = df['datumBeginMeting'].dt.date
    # df['StartTime'] = df['datumBeginMeting'].dt.time
    df['datumEindeMeting'] = pd.to_datetime(df['datumEindeMeting'])
    # df['endDate'] = df['datumEindeMeting'].dt.date
    # df['endTime'] = df['datumEindeMeting'].dt.time
    # df = df.drop(columns=['datumBeginMeting','datumEindeMeting'])
    df.to_parquet('../data/cleanedData/'+ substance +'.parquet', index=False)
    pass

In [29]:
data_files = [
    ('../data/Ammonium_measurements.parquet', 'ammonium'),
    ('../data/Nitrate_measurements.parquet', 'nitrate'),
    ('../data/Oxygen_A.parquet', 'oxygenA'),
    ('../data/Oxygen_B.parquet', 'oxygenB'),
    ('../data/Phosphate_measurements.parquet', 'phosphate')
]

for file_path, var_name in data_files:
    df = pd.read_parquet(file_path)
    cleanData(df, var_name)

In [4]:
with open('../data/Total_influent_flow_WWTP_Ede_2021_minute_data.csv', 'r') as file:
    lines = file.readlines()

with open('../data/cleanedData/total.csv', 'w') as file:
    for line in lines:
        modified_line = line.replace(';60000', '').replace(';',',')
        file.write(modified_line)

finalTotal = pd.read_csv('../data/cleanedData/total.csv')
finalTotal['DateTime'] = pd.to_datetime(finalTotal['DateTime'], dayfirst=True)
# finalTotal['Date'] = finalTotal['DateTime'].dt.date
# finalTotal['Time'] = finalTotal['DateTime'].dt.time
# finalTotal = finalTotal.drop(columns=['DateTime'])
finalTotal.to_csv('../data/cleanedData/total.csv', index=False)

C:\Users\jessi\AppData\Local\Temp\ipykernel_17332\3708884311.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  finalTotal = pd.read_csv('../data/cleanedData/total.csv')


In [89]:
ammonium = pd.read_parquet('../data/cleanedData/ammonium.parquet')
ammonium.head()

duplicates = ammonium[ammonium.duplicated(subset='datumEindeMeting', keep=False)].index.tolist()
# print(ammonium['datumEindeMeting'] == '2021-10-31 02:00:00')
march = ammonium.loc[ammonium['datumEindeMeting'] == '2021-03-28 03:00:00']
#print(march.index)
#print(duplicates)
ammonium.iloc[[436247,436249]]
#ammonium.iloc[[436248: 436369]]
#ammonium.iloc[[123850, 123851]]

,ammoniumValue,datumEindeMeting
436247,1.001,2021-10-31 01:59:00
436249,0.296,2021-10-31 02:00:00


In [96]:
from datetime import timedelta
startTime = march.index.values[0]
endTime = duplicates[-1]

# check =  ammonium['datumEindeMeting'][123] - timedelta(hours=1)
# print(check)

# for i in range(startTime, endTime):
#     if(i < (endTime-119)):
#         ammonium['datumEindeMeting'][i] = ammonium['datumEindeMeting'][i] - timedelta(hours=1)
#     # elif(not(i % 2 == 0)):
#     #     ammonium['datumEindeMeting'][i] = ammonium['datumEindeMeting'][i] - timedelta(hours=1)
#     elif(i > (endTime-119) and not(i % 2 == 0)):
#         ammonium['datumEindeMeting'][i] = ammonium['datumEindeMeting'][i] - timedelta(hours=1)

for i in range(startTime, endTime):
    if i < (endTime - 119):
        ammonium.loc[i, 'datumEindeMeting'] -= timedelta(hours=1)
    elif i > (endTime - 119) and not (i % 2 == 0):
        ammonium.loc[i, 'datumEindeMeting'] -= timedelta(hours=1)

ammonium.iloc[[123849, 123850]]


,ammoniumValue,datumEindeMeting
123849,3.509,2021-03-28 01:59:00
123850,3.520,2021-03-28 00:00:00
